In [3]:
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import os
import sys
import matplotlib.pyplot as plt
import optuna
sys.path.append('../../')


In [4]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [5]:
dataset = pd.read_csv('../data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


In [6]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

test_users = pd.read_csv('../data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
7939,7942
7940,7943
7941,7944
7942,7945


In [7]:
ICM_df = pd.read_csv('../data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [8]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [9]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [9]:
from MatrixFactorization.IALSRecommender_implicit import IALSRecommender_implicit
#current best parameters. Tune again near these values
#printing MAP:  0.07395163955000743
#{'n_factors': 767, 'regularization': 0.6762816208423306, 'iterations': 69, 'weight': 0.5686851029710857}
URM_train_ICM_all_IALS = sps.vstack([URM_train*(1-0.51668), ICM_all.T*0.51668])
recommender_IALS = IALSRecommender_implicit(URM_train_ICM_all_IALS)
recommender_IALS.fit(n_factors = 601, regularization = 0.8715, iterations=83)
evaluator_validation.evaluateRecommender(recommender_IALS)

Recommender_Base_Class: URM Detected 65 (0.23 %) cold users.
Recommender_Base_Class: URM Detected 2 (0.01 %) cold items.



EvaluatorHoldout: Processed 5655 ( 100.00% ) in 6.37 sec. Users per second: 888


({10: {'ROC_AUC': 0.2021407098648488,
   'PRECISION': 0.04530503978779727,
   'PRECISION_RECALL_MIN_DEN': 0.1596137706482541,
   'RECALL': 0.15472556944985233,
   'MAP': 0.0733634002377843,
   'MRR': 0.1509971509971515,
   'NDCG': 0.11049008174386835,
   'F1': 0.07008775413753789,
   'HIT_RATE': 0.45305039787798407,
   'ARHR': 0.17822477650063898,
   'NOVELTY': 0.005378419254206108,
   'AVERAGE_POPULARITY': 0.11650206705379136,
   'DIVERSITY_MEAN_INTER_LIST': 0.9953399719829346,
   'DIVERSITY_HERFINDAHL': 0.999516396137781,
   'COVERAGE_ITEM': 0.4384985563041386,
   'COVERAGE_ITEM_CORRECT': 0.0673339749759384,
   'COVERAGE_USER': 0.7115892789731975,
   'COVERAGE_USER_CORRECT': 0.22863973826601233,
   'DIVERSITY_GINI': 0.1699308060191316,
   'SHANNON_ENTROPY': 12.260853133926368}},
 'CUTOFF: 10 - ROC_AUC: 0.2021407, PRECISION: 0.0453050, PRECISION_RECALL_MIN_DEN: 0.1596138, RECALL: 0.1547256, MAP: 0.0733634, MRR: 0.1509972, NDCG: 0.1104901, F1: 0.0700878, HIT_RATE: 0.4530504, ARHR: 0.17

In [10]:
from GraphBased.RP3betaRecommender import RP3betaRecommender

URM_train_ICM_all_RP3 = sps.vstack([URM_train*(1-0.289), ICM_all.T*0.289])
recommender_rp3Beta = RP3betaRecommender(URM_train_ICM_all_RP3)
recommender_rp3Beta.fit(topK=430, alpha = 0.47, beta=0.27, implicit = False)
recommender_rp3Beta.URM_train = URM_train
evaluator_validation.evaluateRecommender(recommender_rp3Beta)

RP3betaRecommender: URM Detected 65 (0.23 %) cold users.
RP3betaRecommender: URM Detected 2 (0.01 %) cold items.
EvaluatorHoldout: Processed 5655 ( 100.00% ) in 6.12 sec. Users per second: 923


({10: {'ROC_AUC': 0.19880339775167385,
   'PRECISION': 0.04470380194518017,
   'PRECISION_RECALL_MIN_DEN': 0.15711626177143495,
   'RECALL': 0.15235973526894345,
   'MAP': 0.07246636774859251,
   'MRR': 0.14762031072375942,
   'NDCG': 0.10899131446335691,
   'F1': 0.06912551683756926,
   'HIT_RATE': 0.44703801945181254,
   'ARHR': 0.17386517058930923,
   'NOVELTY': 0.00502350296263928,
   'AVERAGE_POPULARITY': 0.06829007818865841,
   'DIVERSITY_MEAN_INTER_LIST': 0.9941563494870889,
   'DIVERSITY_HERFINDAHL': 0.9993980548187444,
   'COVERAGE_ITEM': 0.5866410009624639,
   'COVERAGE_ITEM_CORRECT': 0.06359961501443696,
   'COVERAGE_USER': 0.7115892789731975,
   'COVERAGE_USER_CORRECT': 0.22561973071599345,
   'DIVERSITY_GINI': 0.25452409174816065,
   'SHANNON_ENTROPY': 12.63674963812917}},
 'CUTOFF: 10 - ROC_AUC: 0.1988034, PRECISION: 0.0447038, PRECISION_RECALL_MIN_DEN: 0.1571163, RECALL: 0.1523597, MAP: 0.0724664, MRR: 0.1476203, NDCG: 0.1089913, F1: 0.0691255, HIT_RATE: 0.4470380, ARHR:

In [11]:
from KNN.ItemKNNScoresHybridNRecommender import ItemKNNScoresHybridNRecommender
URM_train_ICM_all = sps.vstack([URM_train, ICM_all.T])

hybrid = ItemKNNScoresHybridNRecommender(URM_train_ICM_all, [recommender_IALS,recommender_rp3Beta], 2)

ItemKNNScoresHybridMultipleRecommender: URM Detected 65 (0.23 %) cold users.
ItemKNNScoresHybridMultipleRecommender: URM Detected 2 (0.01 %) cold items.
number of recommenders: 2


In [12]:
class Objective(object):
    def __init__(self, URM_train, ICM_all, URM_validation, evaluator_validation, hybrid):
        # Hold this implementation specific arguments as the fields of the class.
        self.URM_train = URM_train
        self.ICM_all = ICM_all
        self.URM_validation = URM_validation
        self.evaluator_validation = evaluator_validation
        self.hybrid = hybrid

        

    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
       
        alpha = trial.suggest_uniform('alpha', 0.1, 0.9)
        beta = trial.suggest_uniform('beta', 0.1, 0.9)
        self.hybrid.fit([alpha,beta])
        result_dict, _ = self.evaluator_validation.evaluateRecommender(self.hybrid)

        
        return result_dict[10]["MAP"]
    
# Execute an optimization by using an `Objective` instance.
study = optuna.create_study(direction='maximize')
study.optimize(Objective(URM_train, ICM_all, URM_validation, evaluator_validation, hybrid), n_trials=2000, n_jobs = -1)

print(study.best_params)

[I 2020-12-28 15:06:08,179] A new study created in memory with name: no-name-42a39565-60dc-47b9-8f72-c74cf012c9e1


entering cycle
entering cycle
entering cycle
entering cycle
entering cycle
entering cycle
entering cycle
entering cycle
(1000, 25975)
(1000, 25975)
(1000, 25975)
(1000, 25975)
entering cycle
entering cycle
entering cycle
entering cycle
entering cycle
entering cycle
entering cycle
entering cycle
(1000, 25975)
(1000, 25975)
(1000, 25975)
(1000, 25975)
entering cycle
entering cycleentering cycle

entering cycle
entering cycle
entering cycle
(1000, 25975)
entering cycle
entering cycle
(1000, 25975)
(1000, 25975)
(1000, 25975)
entering cycle
entering cycle
(1000, 25975)
entering cycle
entering cycle
entering cycle
entering cycle
entering cycle
entering cycle
(1000, 25975)
entering cycle
(1000, 25975)
(1000, 25975)
entering cycle
(1000, 25975)
entering cycle
entering cycle
(1000, 25975)
entering cycle
entering cycle
entering cycle
entering cycle
entering cycle
entering cycle
(1000, 25975)
(655, 25975)
(1000, 25975)
entering cycle
entering cycle
(655, 25975)


[I 2020-12-28 15:06:35,253] Trial 2 finished with value: 0.07463897663233994 and parameters: {'alpha': 0.8550372128393872, 'beta': 0.7666188900603824}. Best is trial 2 with value: 0.07463897663233994.


entering cycle
entering cycle


[I 2020-12-28 15:06:36,606] Trial 1 finished with value: 0.07463897663233994 and parameters: {'alpha': 0.13354723008634065, 'beta': 0.22543267737151923}. Best is trial 2 with value: 0.07463897663233994.


...

entering cycle
entering cycle
entering cycle
entering cycle
(655, 25975)
(1000, 25975)


[I 2020-12-28 19:07:54,500] Trial 1998 finished with value: 0.0789203858083166 and parameters: {'alpha': 0.11352500847502914, 'beta': 0.838301225935363}. Best is trial 1784 with value: 0.07927296611738295.


entering cycle
entering cycle
(655, 25975)


[I 2020-12-28 19:07:56,163] Trial 1999 finished with value: 0.07889312789169087 and parameters: {'alpha': 0.11488833768969714, 'beta': 0.7906294132988211}. Best is trial 1784 with value: 0.07927296611738295.


{'alpha': 0.10065566706190891, 'beta': 0.7286916556775159}
